In [1]:
import sys
sys.path.append("..")
from train_by_reconnect.LaPerm import LaPermTrainLoop
from train_by_reconnect.weight_utils import random_prune
from train_by_reconnect.viz_utils import Profiler

In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Dropout, Dense, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
# training parameters
batch_size = 50
epochs = 45

learning_rate = 0.001  # initial learning rate

tsize = 30000  # size of data for getting the train accuracy
vali_freq = 100  # validate per vali_freq batches

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train = np.expand_dims(x_train.astype('float32')/255.0, -1)
x_test = np.expand_dims(x_test.astype('float32')/255.0, -1)


def k_scheduler(epoch):
    return 20


def lr_scheduler(epoch):
    return 1e-3 * 0.95 ** epoch


# data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.10,
    width_shift_range=0.1,
    height_shift_range=0.1)
datagen.fit(x_train)

In [4]:
initializer = 'he_uniform'

model = Sequential()
model.add(Conv2D(32, kernel_size=3, activation='relu',
                 input_shape=(28, 28, 1),
                 kernel_initializer=initializer,
                 bias_initializer=initializer))
model.add(Conv2D(32, kernel_size=3, activation='relu',
                 kernel_initializer=initializer,
                 bias_initializer=initializer))
model.add(Conv2D(32, kernel_size=5, strides=2,
                 padding='same', activation='relu',
                 kernel_initializer=initializer,
                 bias_initializer=initializer))
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size=3, activation='relu',
                 kernel_initializer=initializer,
                 bias_initializer=initializer))
model.add(Conv2D(64, kernel_size=3, activation='relu',
                 kernel_initializer=initializer,
                 bias_initializer=initializer))
model.add(Conv2D(64, kernel_size=5, strides=2,
                 padding='same', activation='relu',
                 kernel_initializer=initializer,
                 bias_initializer=initializer))
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size=4, activation='relu',
                 kernel_initializer=initializer,
                 bias_initializer=initializer))
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax',
                kernel_initializer=initializer,
                bias_initializer=initializer))

# random_prune(model, prune_rate=0.7) # uncomment for random pruning

In [ ]:
loop = LaPermTrainLoop(model=model, loss='sparse_categorical_crossentropy',
                       inner_optimizer=tf.keras.optimizers.Adam(),
                       k_schedule=k_scheduler,
                       lr_schedule=lr_scheduler,
                       skip_bias=False)

loop.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, datagen=datagen,
         validation_data=(x_test, y_test),
         validation_freq=vali_freq,
         tsize=tsize)

In [ ]:
Profiler(model)

In [ ]:
# Visualize train and validation accuracies
import numpy as np
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 6))
plt.plot(loop._history['val accuracy'], label='Validation Accuracy')
plt.plot(loop._history['accuracy'], label='Train Accuracy')
plt.grid(linestyle='--')
plt.xlabel('Epochs', size=15)
plt.ylabel('Accuracy', size=15)
plt.legend(prop={'size': 15})
plt.show()